In [1]:
#import libraries

import scipy.sparse as sps
import numpy as np
import os
import pandas as pd 

URM_all= sps.load_npz("./Processed Matrices/URM_simple_coo_2020.npz").tocsr()
ICM_all= sps.load_npz("./Processed Matrices/ICM_nonweighted_simple_coo_2020.npz").tocsr()

In [2]:
%load_ext Cython

In [3]:
from Notebooks_utils.evaluation_function import evaluate_algorithm
from Base.Similarity.Compute_Similarity_Python import Compute_Similarity_Python
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

np.random.seed(41148)
URM_train, URM_valid = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [4]:
num_user_ratings=[URM_train[u].nnz for u in range(URM_train.shape[0])]

In [5]:
from GraphBased.RP3betaRecommender import RP3betaRecommender
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from MatrixFactorization.IALSRecommender import IALSRecommender
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Base.NonPersonalizedRecommender import TopPop
from KNN.WeightedHybridRecommender import WeightedHybridRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from KNN.WeightedSimilarityHybridRecommender import WeightedSimilarityHybridRecommender
from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender

In [6]:
RP3brecommender= RP3betaRecommender(URM_train)
RP3brecommender.fit(topK=1500, alpha=0.414615342821205, beta=0.17823717015919388, normalize_similarity=False)

ItemKNNCBFrecommender=ItemKNNCBFRecommender(URM_train, ICM_all)
ItemKNNCBFrecommender.fit(topK= 227, shrink=664, similarity='cosine', normalize=False, feature_weighting='BM25')

ItemKNNCFrecommender=ItemKNNCFRecommender(URM_train)
ItemKNNCFrecommender.fit(topK=1500, shrink=52, similarity='tversky', normalize=True, tversky_alpha=0.5985566775975192, tversky_beta=1.6463076214540535)

similarities=[RP3brecommender.W_sparse, ItemKNNCBFrecommender.W_sparse, ItemKNNCFrecommender.W_sparse]
HybridSimilarityrecommender= WeightedSimilarityHybridRecommender(URM_train, similarities)
HybridSimilarityrecommender.fit(np.array([0.5,0.4,0.1]), topK=1000)

RP3betaRecommender: URM Detected 82 (1.03 %) cold users.
RP3betaRecommender: URM Detected 2372 (9.13 %) cold items.
ItemKNNCBFRecommender: URM Detected 82 (1.03 %) cold users.
ItemKNNCBFRecommender: URM Detected 2372 (9.13 %) cold items.
Similarity column 25975 ( 100 % ), 4006.32 column/sec, elapsed time 0.11 min
ItemKNNCFRecommender: URM Detected 82 (1.03 %) cold users.
ItemKNNCFRecommender: URM Detected 2372 (9.13 %) cold items.
Similarity column 25975 ( 100 % ), 5113.56 column/sec, elapsed time 0.08 min
WeightedSimilarityHybridRecommender: URM Detected 82 (1.03 %) cold users.
WeightedSimilarityHybridRecommender: URM Detected 2372 (9.13 %) cold items.


In [7]:
IALSrecommender= IALSRecommender(URM_train)
IALSrecommender.fit(alpha = 49.99999999999999, epochs=10, num_factors=200, confidence_scaling="linear",epsilon=0.002936450234353087, reg=0.01)

IALSRecommender: URM Detected 82 (1.03 %) cold users.
IALSRecommender: URM Detected 2372 (9.13 %) cold items.
IALSRecommender: Epoch 1 of 10. Elapsed time 20.40 sec
IALSRecommender: Epoch 2 of 10. Elapsed time 41.61 sec
IALSRecommender: Epoch 3 of 10. Elapsed time 1.11 min
IALSRecommender: Epoch 4 of 10. Elapsed time 1.53 min
IALSRecommender: Epoch 5 of 10. Elapsed time 1.97 min
IALSRecommender: Epoch 6 of 10. Elapsed time 2.42 min
IALSRecommender: Epoch 7 of 10. Elapsed time 2.84 min
IALSRecommender: Epoch 8 of 10. Elapsed time 3.27 min
IALSRecommender: Epoch 9 of 10. Elapsed time 3.69 min
IALSRecommender: Epoch 10 of 10. Elapsed time 4.12 min
IALSRecommender: Terminating at epoch 10. Elapsed time 4.12 min


In [8]:
ElasticNetrecommender= SLIMElasticNetRecommender(URM_train)
ElasticNetrecommender.fit(topK=863, l1_ratio=2.4149318337459575e-05, alpha=0.001)

SLIMElasticNetRecommender: URM Detected 82 (1.03 %) cold users.
SLIMElasticNetRecommender: URM Detected 2372 (9.13 %) cold items.


/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective di

SLIMElasticNetRecommender: Processed 7023 ( 27.04% ) in 5.00 min. Items per second: 23.40


/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective di

SLIMElasticNetRecommender: Processed 14287 ( 55.00% ) in 10.00 min. Items per second: 23.81


/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective di

SLIMElasticNetRecommender: Processed 21488 ( 82.73% ) in 15.00 min. Items per second: 23.87


/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective di

SLIMElasticNetRecommender: Processed 25975 ( 100.00% ) in 17.90 min. Items per second: 24.18


/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(


In [9]:
TopPoprecommender= TopPop(URM_train)
TopPoprecommender.fit()

TopPopRecommender: URM Detected 82 (1.03 %) cold users.
TopPopRecommender: URM Detected 2372 (9.13 %) cold items.


In [10]:
def WeightGenerator(length,total_sum=100):
    if length == 1:
        yield (total_sum,)
    else:
        for value in range(0,total_sum + 1, 10):
            for permutation in WeightGenerator(length - 1, total_sum - value):
                yield (value,) + permutation

In [11]:
popRec=[HybridSimilarityrecommender,IALSrecommender,ElasticNetrecommender]
unpopRec=[HybridSimilarityrecommender,IALSrecommender,ElasticNetrecommender]

In [14]:
recommender=WeightedHybridRecommender(URM_train, popRec, unpopRec, TopPoprecommender)
evaluator_validation = EvaluatorHoldout(URM_valid, cutoff_list=[10])
resultDF= pd.DataFrame({"threshold":[],"weights1":[],"weights2":[],"MAP":[]})

weights2=[100,0,0]

for threshold in range(10,16):
    for weights1 in list(WeightGenerator(len(popRec))):

        recommender.fit(np.array(weights1)/100, np.array(weights2)/100, threshold)
        MAP,_=evaluator_validation.evaluateRecommender(recommender)
        MAP=MAP[10]["MAP"]
        resultDF=resultDF.append({"threshold":[threshold],"weights1":[weights1],"weights2":[weights2],"MAP":[MAP]},ignore_index=True)
        print(threshold, weights1, weights2, MAP)
    
resultDF_sorted=resultDF.sort_values(by=['MAP'],ascending=False)
resultDF_sorted.to_csv("Weighted_Hybrid_popular_results_sorted.csv")

WeightedHybridRecommender: URM Detected 82 (1.03 %) cold users.
WeightedHybridRecommender: URM Detected 2372 (9.13 %) cold items.
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 21.68 sec. Users per second: 261
10 (0, 0, 100) [100, 0, 0] 0.06848136455198803
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 21.81 sec. Users per second: 259
10 (0, 10, 90) [100, 0, 0] 0.06976309265153875
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 21.77 sec. Users per second: 260
10 (0, 20, 80) [100, 0, 0] 0.07022350555166512
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 21.75 sec. Users per second: 260
10 (0, 30, 70) [100, 0, 0] 0.07040975291739723
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 21.70 sec. Users per second: 261
10 (0, 40, 60) [100, 0, 0] 0.07024176020358057
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 21.66 sec. Users per second: 261
10 (0, 50, 50) [100, 0, 0] 0.06983787135343981
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 21.73 sec. Users per second: 260
10 (0, 60, 40) [100, 

In [15]:
resultDF= pd.DataFrame({"threshold":[],"weights1":[],"weights2":[],"MAP":[]})
weights1=[100,0,0]

for threshold in range(10,16):
    for weights2 in list(WeightGenerator(len(unpopRec))):

        recommender.fit(np.array(weights1)/100, np.array(weights2)/100, threshold)
        MAP,_=evaluator_validation.evaluateRecommender(recommender)
        MAP=MAP[10]["MAP"]
        resultDF=resultDF.append({"threshold":[threshold],"weights1":[weights1],"weights2":[weights2],"MAP":[MAP]},ignore_index=True)
        print(threshold, weights1, weights2, MAP)
    
resultDF_sorted=resultDF.sort_values(by=['MAP'],ascending=False)
resultDF_sorted.to_csv("Weighted_Hybrid_unpopular_results_sorted.csv")

EvaluatorHoldout: Processed 5657 ( 100.00% ) in 21.18 sec. Users per second: 267
10 [100, 0, 0] (0, 0, 100) 0.05990579632130441
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 21.73 sec. Users per second: 260
10 [100, 0, 0] (0, 10, 90) 0.06220987653764254
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 21.74 sec. Users per second: 260
10 [100, 0, 0] (0, 20, 80) 0.06315775400929582
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 22.42 sec. Users per second: 252
10 [100, 0, 0] (0, 30, 70) 0.06393940253577386
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 22.52 sec. Users per second: 251
10 [100, 0, 0] (0, 40, 60) 0.06449559760429875
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 22.23 sec. Users per second: 255
10 [100, 0, 0] (0, 50, 50) 0.06468991417955293
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 23.40 sec. Users per second: 242
10 [100, 0, 0] (0, 60, 40) 0.06457191292998904
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 22.45 sec. Users per second: 252
10 [100, 0, 0] (0, 70, 

In [16]:
RP3brecommender_full= RP3betaRecommender(URM_all)
RP3brecommender_full.fit(topK=770, alpha=0.414615342821205, beta=0.17823717015919388, normalize_similarity=False)

ItemKNNCBFrecommender_full=ItemKNNCBFRecommender(URM_all, ICM_all)
ItemKNNCBFrecommender_full.fit(topK= 227, shrink=664, similarity='cosine', normalize=False, feature_weighting='BM25')

ItemKNNCFrecommender_full=ItemKNNCFRecommender(URM_all)
ItemKNNCFrecommender_full.fit(topK=210, shrink=52, similarity='tversky', normalize=True, tversky_alpha=0.5985566775975192, tversky_beta=1.6463076214540535)

similarities_full=[RP3brecommender_full.W_sparse, ItemKNNCBFrecommender_full.W_sparse, ItemKNNCFrecommender_full.W_sparse]
HybridSimilarityrecommender= WeightedSimilarityHybridRecommender(URM_all, similarities_full)
HybridSimilarityrecommender.fit(np.array([0.5,0.4,0.1]), topK=1000)


IALSrecommender= IALSRecommender(URM_all)
IALSrecommender.fit(alpha = 49.99999999999999, epochs=10, num_factors=200, confidence_scaling="linear",epsilon=0.002936450234353087, reg=0.01)

ElasticNetrecommender= SLIMElasticNetRecommender(URM_all)
ElasticNetrecommender.fit(topK=863, l1_ratio=2.4149318337459575e-05, alpha=0.001)

TopPoprecommender= TopPop(URM_all)
TopPoprecommender.fit()

RP3betaRecommender: URM Detected 1079 (4.15 %) cold items.
ItemKNNCBFRecommender: URM Detected 1079 (4.15 %) cold items.
Similarity column 25975 ( 100 % ), 4123.41 column/sec, elapsed time 0.10 min
ItemKNNCFRecommender: URM Detected 1079 (4.15 %) cold items.
Similarity column 25975 ( 100 % ), 5209.72 column/sec, elapsed time 0.08 min
WeightedSimilarityHybridRecommender: URM Detected 1079 (4.15 %) cold items.
IALSRecommender: URM Detected 1079 (4.15 %) cold items.
IALSRecommender: Epoch 1 of 10. Elapsed time 20.14 sec
IALSRecommender: Epoch 2 of 10. Elapsed time 40.08 sec
IALSRecommender: Epoch 3 of 10. Elapsed time 1.02 min
IALSRecommender: Epoch 4 of 10. Elapsed time 1.45 min
IALSRecommender: Epoch 5 of 10. Elapsed time 1.85 min
IALSRecommender: Epoch 6 of 10. Elapsed time 2.27 min
IALSRecommender: Epoch 7 of 10. Elapsed time 2.69 min
IALSRecommender: Epoch 8 of 10. Elapsed time 3.10 min
IALSRecommender: Epoch 9 of 10. Elapsed time 3.54 min
IALSRecommender: Epoch 10 of 10. Elapsed tim

/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective di

SLIMElasticNetRecommender: Processed 8166 ( 31.44% ) in 5.00 min. Items per second: 27.22


/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective di

SLIMElasticNetRecommender: Processed 16210 ( 62.41% ) in 10.00 min. Items per second: 27.01


/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective di

SLIMElasticNetRecommender: Processed 23820 ( 91.70% ) in 15.00 min. Items per second: 26.46


/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:512: ConvergenceWarning: Objective di

SLIMElasticNetRecommender: Processed 25975 ( 100.00% ) in 16.57 min. Items per second: 26.13
TopPopRecommender: URM Detected 1079 (4.15 %) cold items.


NameError: name 'IALSrecommenderElasticNetrecommender' is not defined

In [19]:
popRec=[HybridSimilarityrecommender,IALSrecommender,ElasticNetrecommender]
unpopRec=[HybridSimilarityrecommender,IALSrecommender,ElasticNetrecommender]


recommender_full=WeightedHybridRecommender(URM_all, popRec, unpopRec, TopPoprecommender)
recommender_full.fit(np.array([0.5,0.2,0.3]),np.array([0.9,0.1,0]) , 15)

WeightedHybridRecommender: URM Detected 1079 (4.15 %) cold items.


In [20]:
rec_df=pd.DataFrame({"user_id":[],"item_list":[]})
target_user_list=list(pd.read_csv("./Data/2020/data_target_users_test.csv").user_id)

for uid in target_user_list:
    rec_list=map(str,recommender_full.recommend(uid)[:10])
    rec_df=rec_df.append({"user_id":uid,"item_list":' '.join(rec_list)},ignore_index=True)

    
rec_df=rec_df.astype({"user_id":'int32',"item_list":'str'})
rec_df=rec_df.set_index('user_id')
rec_df.to_csv("Weighted_Hybrid_recommendations.csv")